In [1]:
# Copyright (C) 2020-2022 TU Darmstadt
# SPDX-License-Identifier: Apache-2.0

# -----------------------------------------------------------
# Primary author: Phillip Rieger <phillip.rieger@trust.tu-darmstadt.de>
# Co-authored-by: Torsten Krauss <torsten.krauss@uni-wuerzburg.de>
# ------------------------------------------------------------

import argparse
import os
import pickle
import time
import warnings
from copy import deepcopy
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import torch.optim as optim
from torchvision import transforms, datasets
from sklearn.cluster import AgglomerativeClustering, DBSCAN

from CrowdGuardClientValidation import CrowdGuardClientValidation
from openfl.experimental.interface import Aggregator, Collaborator, FLSpec
from openfl.experimental.placement import aggregator, collaborator
from openfl.experimental.runtime import LocalRuntime
from cifar10_crowdguard import MEAN, STD_DEV, poison_data, seed_random_generators
from cifar10_crowdguard import BATCH_SIZE_TRAIN, BATCH_SIZE_TEST, Net, test, default_optimizer
from cifar10_crowdguard import FederatedFlow
warnings.filterwarnings("ignore")

Aggregator step "start" registered
Collaborator step "train" registered
Aggregator step "fed_avg_aggregation" registered
Aggregator step "collect_models" registered
Collaborator step "local_validation" registered
Aggregator step "defend" registered
Aggregator step "end" registered


In [2]:
TOTAL_CLIENT_NUMBER = 4
PMR = 0.25
NUMBER_OF_MALICIOUS_CLIENTS = max(1, int(TOTAL_CLIENT_NUMBER * PMR)) if PMR > 0 else 0
NUMBER_OF_BENIGN_CLIENTS = TOTAL_CLIENT_NUMBER - NUMBER_OF_MALICIOUS_CLIENTS
NUMBER_OF_ROUNDS = 10

In [3]:
class CommandLineArgumentSimulator:
    
    def __init__(self):
        self.test_dataset_ratio = 0.4
        self.train_dataset_ratio = 0.4
        self.log_dir = 'test_debug'
        self.comm_round = NUMBER_OF_ROUNDS
        self.flow_internal_loop_test=False
        self.optimizer_type = 'SGD'
        
args = CommandLineArgumentSimulator()

In [4]:
aggregator_object = Aggregator()
aggregator_object.private_attributes = {}
collaborator_names = [f'benign_{i:02d}' for i in range(NUMBER_OF_BENIGN_CLIENTS)] + [f'malicious_{i:02d}' for i in range(NUMBER_OF_MALICIOUS_CLIENTS)]    
collaborators = [Collaborator(name=name) for name in collaborator_names]
if torch.cuda.is_available():
    device = torch.device(
        "cuda:1"
        )  # This will enable Ray library to reserve available GPU(s) for the task
else:
    device = torch.device("cpu")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(MEAN, STD_DEV),])

cifar_train = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
cifar_train = [x for x in cifar_train]
cifar_test = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
cifar_test = [x for x in cifar_test]
X = torch.stack([x[0] for x in cifar_train] + [x[0] for x in cifar_test])
Y = torch.LongTensor(np.stack(np.array([x[1] for x in cifar_train] + [x[1] for x in cifar_test])))

Files already downloaded and verified
Files already downloaded and verified


In [5]:
seed_random_generators(0)
shuffled_indices = np.arange(X.shape[0])
np.random.shuffle(shuffled_indices)

N_total_samples = len(cifar_test) + len(cifar_train)
train_dataset_size = int(N_total_samples * args.train_dataset_ratio)
test_dataset_size = int(N_total_samples * args.test_dataset_ratio)
X = X[shuffled_indices]
Y = Y[shuffled_indices]

train_dataset_data = X[:train_dataset_size]
train_dataset_targets = Y[:train_dataset_size]

test_dataset_data = X[train_dataset_size:train_dataset_size + test_dataset_size]
test_dataset_targets = Y[train_dataset_size:train_dataset_size + test_dataset_size]
print(f"Dataset info (total {N_total_samples}): train - {test_dataset_targets.shape[0]}, "
          f"test - {test_dataset_targets.shape[0]}, ")


Dataset info (total 60000): train - 24000, test - 24000, 


In [6]:
for idx, collab in enumerate(collaborators):
    # construct the training and test and population dataset
    benign_training_X = train_dataset_data[idx::len(collaborators)]
    benign_training_Y = train_dataset_targets[idx::len(collaborators)]
    
    if 'malicious' in collab.name:
        local_train_data, local_train_targets = poison_data(benign_training_X, benign_training_Y)
    else:
        local_train_data, local_train_targets = benign_training_X, benign_training_Y
    

    local_test_data = test_dataset_data[idx::len(collaborators)]
    local_test_targets = test_dataset_targets[idx::len(collaborators)]
    

    poison_test_data, poison_test_targets = poison_data(local_test_data, local_test_targets,
                                                        pdr=1.0)

    collab.private_attributes = {
        "train_loader": torch.utils.data.DataLoader(
            TensorDataset(local_train_data, local_train_targets),
            batch_size=BATCH_SIZE_TRAIN, shuffle=True
            ),
        "test_loader": torch.utils.data.DataLoader(
            TensorDataset(local_test_data, local_test_targets),
            batch_size=BATCH_SIZE_TEST, shuffle=False
            ),
        "backdoor_test_loader": torch.utils.data.DataLoader(
            TensorDataset(poison_test_data, poison_test_targets),
            batch_size=BATCH_SIZE_TEST, shuffle=False
            ),
        }

In [7]:
pretrained_weights = torch.load('pretrained_cifar.pt', map_location=device)
test_model = Net().to(device)
test_model.load_state_dict(pretrained_weights)
test(test_model, collab.private_attributes['train_loader'], device, test_train='Train')
test(test_model, collab.private_attributes['test_loader'], device)
test(test_model, collab.private_attributes['backdoor_test_loader'], device, mode='Backdoor')

Benign Train set: Avg. loss: 3.3843570884237897, Accuracy: 2083/6000 (34.716665744781494%)
Benign Test set: Avg. loss: 0.9973345994949341, Accuracy: 3768/6000 (62.800002098083496%)
Backdoor Test set: Avg. loss: 5.72957197825114, Accuracy: 325/6000 (5.416666716337204%)


0.05416666716337204

In [8]:
local_runtime = LocalRuntime(aggregator=aggregator_object, collaborators=collaborators)

print(f"Local runtime collaborators = {local_runtime.collaborators}")

# change to the internal flow loop
model = Net()
model.load_state_dict(pretrained_weights)
top_model_accuracy = 0
optimizers = {
    collaborator.name: default_optimizer(model, optimizer_type=args.optimizer_type)
    for collaborator in collaborators
    }
flflow = FederatedFlow(
    model,
    optimizers,
    device,
    args.comm_round,
    top_model_accuracy,
    NUMBER_OF_MALICIOUS_CLIENTS / TOTAL_CLIENT_NUMBER,
    'CrowdGuard'
    )

flflow.runtime = local_runtime
flflow.run()

Local runtime collaborators = ['benign_00', 'benign_01', 'benign_02', 'malicious_00']
####################
Round 0...
####################

Calling start
Performing initialization for model
Sending state from aggregator to collaborators

Calling train
####################
Performing model training for collaborator benign_00 in round 0
Benign Train set: Avg. loss: 0.9885769543495584, Accuracy: 3790/6000 (63.16666603088379%)
Benign Test set: Avg. loss: 1.001497248808543, Accuracy: 3761/6000 (62.68333196640015%)
Backdoor Test set: Avg. loss: 5.7991689046223955, Accuracy: 330/6000 (5.499999970197678%)
Benign Train set: Avg. loss: 1.0336276793733556, Accuracy: 3636/6000 (60.600000619888306%)
Benign Test set: Avg. loss: 1.178040087223053, Accuracy: 3368/6000 (56.13332986831665%)
Backdoor Test set: Avg. loss: 5.754194895426433, Accuracy: 266/6000 (4.43333312869072%)
Should transfer from train to collect_models

Calling train
####################
Performing model training for collaborator beni

Backdoor Test set: Avg. loss: 5.272974332173665, Accuracy: 318/6000 (5.299999937415123%)
Benign Train set: Avg. loss: 0.5982799826467291, Accuracy: 4669/6000 (77.8166651725769%)
Benign Test set: Avg. loss: 1.2914982438087463, Accuracy: 3081/6000 (51.34999752044678%)
Backdoor Test set: Avg. loss: 0.022719704546034336, Accuracy: 5992/6000 (99.86666440963745%)
Scale Model by 4.0
Should transfer from train to collect_models

Calling collect_models
Sending state from aggregator to collaborators

Calling local_validation
Performing model validation for collaborator benign_00 in round 1
Distance: cosine, use y 2: [2.759627751757632, 0.6747370290125163]
Distance: cosine, use x 2: [16.107230146795576, 0.6747370290125163]
Distance: cosine, use y 1: [10.954451150103317]
Distance: cosine, use x 2: [10.954451150103317, 0.0]
Distance: euclid, use y 2: [0.14883609059085856, 0.02115055331254201]
Distance: euclid, use x 2: [20.601495811084128, 0.02115055331254112]
Distance: euclid, use y 1: [10.9544511

Distance: cosine, use y 2: [6.51663607253947, 0.738155916099045]
Distance: cosine, use x 2: [13.852920216708299, 0.738155916099045]
Distance: cosine, use y 1: [10.954451150103315]
Distance: cosine, use x 2: [10.954451150103319, 0.0]
Distance: euclid, use y 2: [1.0858903795599173, 2.7595749715141165]
Distance: euclid, use x 2: [16.19100737219361, 1.0858903795599173]
Distance: euclid, use y 1: [10.954451150103319]
Distance: euclid, use x 2: [10.954451150103349, 0.0]
Suspicious Models detected by 3: [1]
Should transfer from local_validation to defend

Calling defend
Agglomerative Clustering: {0: array([0, 1, 2]), 1: array([3])}
DBScan Input: [0 1 2]
DBScan Clustering: [0 1 2]
Negatives: [0, 1, 2]
Finished round 3/10
Sending state from aggregator to collaborators

Calling train
####################
Performing model training for collaborator benign_00 in round 3
Benign Train set: Avg. loss: 0.8878433810586625, Accuracy: 4102/6000 (68.36666464805603%)
Benign Test set: Avg. loss: 0.9929341574

Benign Test set: Avg. loss: 1.0727848609288533, Accuracy: 3640/6000 (60.66666841506958%)
Backdoor Test set: Avg. loss: 4.833694140116374, Accuracy: 404/6000 (6.733333319425583%)
Should transfer from train to collect_models

Calling train
####################
Performing model training for collaborator malicious_00 in round 4
Benign Train set: Avg. loss: 3.337863416113752, Accuracy: 2106/6000 (35.100001096725464%)
Benign Test set: Avg. loss: 0.9814265072345734, Accuracy: 3856/6000 (64.26666378974915%)
Backdoor Test set: Avg. loss: 5.6613701184590655, Accuracy: 325/6000 (5.416666716337204%)
Benign Train set: Avg. loss: 0.550559540299025, Accuracy: 4774/6000 (79.56666350364685%)
Benign Test set: Avg. loss: 1.199839214483897, Accuracy: 3231/6000 (53.850001096725464%)
Backdoor Test set: Avg. loss: 0.013157528980324665, Accuracy: 5990/6000 (99.83333349227905%)
Scale Model by 4.0
Should transfer from train to collect_models

Calling collect_models
Sending state from aggregator to collaborators

Suspicious Models detected by 2: [3]
Should transfer from local_validation to defend

Calling local_validation
Performing model validation for collaborator malicious_00 in round 5
Distance: cosine, use y 2: [1.5875451563157443, 6.072490248262797]
Distance: cosine, use x 2: [13.76536491353733, 1.587545156315744]
Distance: cosine, use y 1: [10.95445115010333]
Distance: cosine, use x 2: [10.95445115010333, 0.0]
Distance: euclid, use y 2: [2.9658612747459934, 0.832694164710559]
Distance: euclid, use x 2: [17.05189518013359, 0.8326941647105595]
Distance: euclid, use y 1: [10.954451150103292]
Distance: euclid, use x 2: [10.954451150103372, 0.0]
Suspicious Models detected by 3: [0, 2]
Should transfer from local_validation to defend

Calling defend
Agglomerative Clustering: {0: array([0, 1, 2]), 1: array([3])}
DBScan Input: [0 1 2]
DBScan Clustering: [0 1 2]
Negatives: [0, 1, 2]
Finished round 6/10
Sending state from aggregator to collaborators

Calling train
####################
Performing mo

Backdoor Test set: Avg. loss: 6.572487751642863, Accuracy: 275/6000 (4.583333432674408%)
Benign Train set: Avg. loss: 0.8441252261400223, Accuracy: 4159/6000 (69.31666731834412%)
Benign Test set: Avg. loss: 1.1365716060002644, Accuracy: 3499/6000 (58.31666588783264%)
Backdoor Test set: Avg. loss: 7.446105162302653, Accuracy: 76/6000 (1.2666666880249977%)
Should transfer from train to collect_models

Calling train
####################
Performing model training for collaborator malicious_00 in round 7
Benign Train set: Avg. loss: 3.7784291537518198, Accuracy: 2079/6000 (34.65000092983246%)
Benign Test set: Avg. loss: 0.9853859345118204, Accuracy: 3836/6000 (63.93333077430725%)
Backdoor Test set: Avg. loss: 6.557899475097656, Accuracy: 291/6000 (4.849999770522118%)
Benign Train set: Avg. loss: 0.5669700073751998, Accuracy: 4784/6000 (79.7333300113678%)
Benign Test set: Avg. loss: 1.1976680755615234, Accuracy: 3299/6000 (54.98332977294922%)
Backdoor Test set: Avg. loss: 0.04168200430770715

Distance: cosine, use y 2: [0.3178463131074398, 5.67619041302248]
Distance: cosine, use x 2: [15.740257140391499, 0.31784631310744027]
Distance: cosine, use y 1: [10.95445115010334]
Distance: cosine, use x 2: [10.95445115010333, 0.0]
Distance: euclid, use y 2: [0.06679420027461447, 0.009586160307017089]
Distance: euclid, use x 2: [20.632805803633033, 0.0095861603070162]
Distance: euclid, use y 1: [10.954451150103338]
Distance: euclid, use x 2: [10.954451150103328, 0.0]
Suspicious Models detected by 2: [3]
Should transfer from local_validation to defend

Calling local_validation
Performing model validation for collaborator malicious_00 in round 8
Distance: cosine, use y 2: [1.5848936880939157, 4.9747467633275715]
Distance: cosine, use x 2: [14.425323893096719, 1.5848936880939157]
Distance: cosine, use y 1: [10.95445115010334]
Distance: cosine, use x 2: [10.954451150103333, 0.0]
Distance: euclid, use y 2: [0.35019439197558766, 8.02170037454417]
Distance: euclid, use x 2: [12.080158298652